`pip install scikit-learn` : 사이킷 런 설치

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score

# 데이터셋 로드
dataset = load_dataset("imdb") # imdb : 영화의 리뷰 텍스트와 그에 대한 감정 레이블이 포함된 데이터셋. 영화 리뷰 감성 이진분류에 주로 사용
# imdb 레이블 : 0 - 부정적 리뷰, 1 - 긍정적 리뷰

# 테스트 데이터셋 설정
test_dataset = dataset["test"].shuffle(seed=42).select(range(500))
 
# 토크나이저로 입력 토큰화 진행
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# bert 모델이 이해할 수 있게 텍스트를 토큰형태로 변환
def tokenize_function(examples):
    return tokenizer(
        examples["text"], # imdb 데이터셋에서 리뷰를 가져옴
        padding="max_length", # 모든 텍스트를 최대길이로 패딩 - 모든 입력이 동일한 길이를 갖도록 함
        truncation=True # 텍스트의 길이가 최대길이를 초과할 시 잘라냄
    )

# 데이터셋의 각 샘플에 토크나이즈펑션 진행. 
test_dataset = test_dataset.map(
    tokenize_function, 
    batched=True # 배치 단위로 함수를 적용 여러 샘플 한 번에 처리. 속도 높이고 메모리 최적화 됨
    )
 
# 데이터셋을 특정 형식으로 변환
test_dataset.set_format(
    type="torch", # 파이토치 텐서로 변환
    columns=['input_ids','attention_mask','label'] # 버트 모델에 필요한 데이터 컬럼만 선택
)

# Bert 모델 로드
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

# 모델을 평가 모드로 바꿈
model.eval() 

# 예측 및 평가
all_preds = []
all_labels = []

for batch in torch.utils.data.DataLoader( # DataLoader : 데이터셋을 배치단위로 로드
    test_dataset,  # 평가에 사용할 데이터셋
    batch_size=8   # 배치 사이즈
    ):
    with torch.no_grad(): # 그래디언트를 계산하지 않음 -> 추론을 하기위해 사용
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']) # 모델에 대한 아웃풋 생성
    logits = outputs.logits 
    preds = np.argmax(logits.numpy(),axis=1)
    all_preds.extend(preds)
    all_labels.extend(batch['label'].numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)    
print(f"Accuracy without fine-tuning: {accuracy:.4f}")

/opt/anaconda3/envs/AI_8/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy without fine-tuning: 0.5080


# 파인튜닝
: 

In [2]:


from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [3]:
# IMDb 데이터셋 로드
dataset = load_dataset("imdb")

# 훈련 및 테스트 데이터셋 분리
train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))  # 1000개 샘플로 축소
test_dataset = dataset['test'].shuffle(seed=42).select(range(500))  # 500개 샘플로 축소

In [4]:
# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터셋 토크나이징 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# 데이터셋 토크나이징 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 모델 입력으로 사용하기 위해 데이터셋 포맷 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/opt/anaconda3/envs/AI_8/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# BERT 모델 로드
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


# 훈련 인자 설정
training_args = TrainingArguments( # 모델훈련시 필요한 설정 정리
    output_dir='./results', # 훈련중 생성된 모델 파일 및 로그 저장 디렉토리
    num_train_epochs=3, # 훈련 에포크 수
    per_device_train_batch_size=8, # 각 디바이스의 사용할 배치 크기 : 8개 샘플
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch", # 평가 전략 : 에포크가 끝날 때 마다 평가 진행
    save_steps=10_000, # 10000 스텝마다 모델 저장
    save_total_limit=2, # 저장할 최대 체크포인트의 개수
)

# 트레이너 설정
trainer = Trainer( # 모델 훈련, 평가, 예측 간단히 수행할 수 있는 클래스
    model=model, # 훈련시킬 모델
    args=training_args, # 학습을 위한 설정 값
    train_dataset=train_dataset, 
    eval_dataset=test_dataset,
)

# 모델 훈련
trainer.train()
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: Accelerator.__init__() got an unexpected keyword argument 'dispatch_batches'

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score

# 평가 지표 함수 정의
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)  # 예측된 클래스
    labels = p.label_ids  # 실제 레이블
    acc = accuracy_score(labels, preds)  # 정확도 계산
    return {'accuracy': acc}

# 이미 훈련된 트레이너에 compute_metrics를 추가하여 평가
trainer.compute_metrics = compute_metrics

# 모델 평가 및 정확도 확인
eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

NameError: name 'trainer' is not defined